In [36]:
import pandas as pd
import numpy as np

In [37]:
# 读取数据至dataframe内
dataset = pd.read_csv('./Market_Basket_Optimisation.csv', header = None)
dataset.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# 使用空字节填充NaN
dataset = dataset.fillna('')
dataset.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,,,,,,,,,,,,,,,,,
2,chutney,,,,,,,,,,,,,,,,,,,
3,turkey,avocado,,,,,,,,,,,,,,,,,,
4,mineral water,milk,energy bar,whole wheat rice,green tea,,,,,,,,,,,,,,,


In [39]:
# 将每一行数据组合成一列，用逗号分隔
dataset_combined = dataset[0]
for i in range(1, len(dataset.columns)):
    dataset_combined = dataset_combined + ',' + dataset[i]
dataset_combined = pd.DataFrame(dataset_combined)
dataset_combined.columns = ['transactions']
dataset_combined.head(10)

,transactions
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs,,,,,,,,,,,,,,,,,"
2,"chutney,,,,,,,,,,,,,,,,,,,"
3,"turkey,avocado,,,,,,,,,,,,,,,,,,"
4,"mineral water,milk,energy bar,whole wheat rice..."
5,"low fat yogurt,,,,,,,,,,,,,,,,,,,"
6,"whole wheat pasta,french fries,,,,,,,,,,,,,,,,,,"
7,"soup,light cream,shallot,,,,,,,,,,,,,,,,,"
8,"frozen vegetables,spaghetti,green tea,,,,,,,,,..."
9,"french fries,,,,,,,,,,,,,,,,,,,"


In [40]:
# 将购物物品转化为one hot编码
dataset = dataset_combined['transactions'].str.get_dummies(',')
dataset.head(5)

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [41]:
# 使用mlxtend提炼数据中的关联规则
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(dataset, min_support=0.015, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by="support" , ascending=False)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rules = rules.sort_values(by="lift" , ascending=False)
print("频繁项集：", frequent_itemsets)
print("关联规则：", rules)

频繁项集：       support                                 itemsets
36   0.238368                          (mineral water)
14   0.179709                                   (eggs)
52   0.174110                              (spaghetti)
18   0.170911                           (french fries)
10   0.163845                              (chocolate)
25   0.132116                              (green tea)
35   0.129583                                   (milk)
26   0.098254                            (ground beef)
23   0.095321                      (frozen vegetables)
42   0.095054                               (pancakes)
3    0.087188                                (burgers)
5    0.081056                                   (cake)
11   0.080389                                (cookies)
17   0.079323                               (escalope)
33   0.076523                         (low fat yogurt)
50   0.071457                                 (shrimp)
55   0.068391                               (tomatoes)
41  

发现香叶&胡椒与牛肉的提升关系最强，这也是符合常理的，因为牛肉一般都需要加胡椒和香叶。
而{香叶&胡椒，牛肉}的组合的支持值不高，只有0.015998，但是香叶&胡椒和牛肉的单独组合的支持值比较高，分别达到了0.49和0.98，说明这两样商品比较受欢迎但是人们一般不会同时购买它们。

In [42]:
# 使用efficient_apriori提炼关联关系
from efficient_apriori import apriori

dataset = pd.read_csv('./Market_Basket_Optimisation.csv', header = None) 
# 将数据存放到transactions中
transactions = []
for i in range(0, dataset.shape[0]):
    temp = []
    for j in range(0, 20):
        if str(dataset.values[i, j]) != 'nan':
           temp.append(str(dataset.values[i, j]))
    transactions.append(temp)
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(transactions, min_support=0.015,  min_confidence=0.2)
print("频繁项集：", itemsets)
print("关联规则：", rules)

频繁项集： {1: {('low fat yogurt',): 574, ('cottage cheese',): 239, ('honey',): 356, ('frozen smoothie',): 475, ('almonds',): 153, ('avocado',): 250, ('salmon',): 319, ('green tea',): 991, ('shrimp',): 536, ('mineral water',): 1788, ('olive oil',): 494, ('vegetables mix',): 193, ('tomato juice',): 228, ('energy drink',): 200, ('eggs',): 1348, ('burgers',): 654, ('meatballs',): 157, ('turkey',): 469, ('energy bar',): 203, ('whole wheat rice',): 439, ('milk',): 972, ('french fries',): 1282, ('whole wheat pasta',): 221, ('soup',): 379, ('light cream',): 117, ('frozen vegetables',): 715, ('spaghetti',): 1306, ('cookies',): 603, ('cooking oil',): 383, ('champagne',): 351, ('chicken',): 450, ('oil',): 173, ('chocolate',): 1229, ('fresh tuna',): 167, ('tomatoes',): 513, ('protein bar',): 139, ('pasta',): 118, ('pepper',): 199, ('red wine',): 211, ('rice',): 141, ('ham',): 199, ('pancakes',): 713, ('white wine',): 124, ('grated cheese',): 393, ('parmesan cheese',): 149, ('fresh bread',): 323, ('esc

可以看到对于数据量比较大的情况efficient_apriori给出的信息比较少